# Mockup Version 1

- Mock-up the workflow from MIS global summit in an environment like EvidenceCare's Epic App. Due January 1.
    - ☑️ List content for our app (Due by Nov 23) TJM
    - ☑️ Define the workflow between user, website, and service (Due by Nov 23) TJM & NS workshop session
    - ☑️ Define interface between UI and service (Due by Nov 30) TJM & NS workshop session
    - ☑️ Include procedures predictor (Due by Dec 14) TJM
    - ☑️ Create a static page layout (Due by Dec 7) NS
    - ☑️ Flask configuration (due by Dec 1) TJM 
    - Have shortest path algorithms produce sentences of node-relationship-node, where you can click on a word and it highlights the node in the graph visualization
    - Tim give Nikesh an explanation of the score value which he can add to the prototype
    - Tim incorporate Nikesh's fetchData and flask code into current code
    - Write & test the code for webpage (Due by Dec 23) TJM & NS
    - Provide instructions, documentation, supporting information to make it (Due by Dec 23) BED
    - 🟢 Create a page on mi1.ai that introduces the POC
    - 🟢 Create a slide that describes what is going on under the hood in the poc
    - 🟢 Create an idealized version of visualization, which can be inserted into a screen-recorded demo right after showing the live graph view
    

In [1]:
import pandas as pd
import time

In [2]:
# import getpass
# password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j','NikeshIsCool'))
session=driver.session()

In [19]:
# Get lists of entities associated with a couple of diseases in MIMIC (diastolic CHF, NSCLC, and urosepsis)
query = '''
MATCH (d)-[total_r:OCCURS_WITH]-(total_f)
WHERE d.cui IN ["C2183328", "C0007131", "C0149801", "C0154298"]
RETURN d.term AS Disease, collect(total_f.cui) as Assoc_Entities, count(total_f.cui) AS Number_of_Assoc_Entities'''
data = session.run(query)
disease_example = pd.DataFrame([dict(record) for record in data])
disease_example

,Disease,Assoc_Entities,Number_of_Assoc_Entities
0,Non-Small Cell Lung Carcinoma,"[C0242379, C0684249, C1839839]",3
1,Sepsis due to urinary tract infection,"[C0264490, C0034186, C0267026, C3714514, C0268...",11
2,diastolic congestive heart failure,"[C0520863, C0043144, C0235480, C0018802, C0546...",19
3,Acute posthaemorrhagic anaemia,"[C1135191, C0497131, C0014591, C0007787, C0019...",5


In [21]:
# Collect lists of CUIs for the disease-associated entities
NSCLC_entities = disease_example['Assoc_Entities'][0]
urosepsis_entities = disease_example['Assoc_Entities'][1]
dCHF_entities = disease_example['Assoc_Entities'][2]
bleed_out_entities = disease_example['Assoc_Entities'][3]


multi_dz_entities = list(set(dCHF_entities + NSCLC_entities + urosepsis_entities + bleed_out_entities))

len(multi_dz_entities)

36

In [14]:
# Get a differential diagnosis for non-rare diseases
def DDx(cui_finding_list):

    query = '''
    MATCH (pos_f)-[given_r:OCCURS_WITH]-(d)-[total_r:OCCURS_WITH]-(total_f)

    // Get a list of diseases which have at least one finding in the list of CUIs for given findings
    WHERE pos_f.cui IN {cui_finding_list}
 AND (d.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"] OR d:D_Icd_Diagnoses)

     // Get lists of positive findings and relationships to positive findings
    WITH d.term AS Disease, d.cui AS Disease_CUI, collect(DISTINCT(pos_f.term)) AS Positive_Findings, collect(DISTINCT(pos_f.cui)) AS Pos_Find_CUIs, collect(DISTINCT(total_r)) AS All_Find_Rel, collect(DISTINCT(total_f.term)) AS all_dz_findings, collect(DISTINCT(total_f.cui)) AS all_dz_CUIs, collect(DISTINCT(given_r)) AS Pos_Find_Rel

    // Get list of relationships to negative findings
    WITH [x IN All_Find_Rel WHERE NOT x IN Pos_Find_Rel] AS Neg_Find_Rel, Pos_Find_Rel, Disease, Disease_CUI, all_dz_findings, all_dz_CUIs, Positive_Findings, Pos_Find_CUIs

    // Get list of negative findings
    WITH [x IN all_dz_findings WHERE NOT x in Positive_Findings] AS Negative_Findings, [x IN all_dz_CUIs WHERE NOT x in Pos_Find_CUIs] AS Neg_Find_CUIs, Neg_Find_Rel, Pos_Find_Rel, Disease, Disease_CUI, Positive_Findings, Pos_Find_CUIs

    UNWIND Pos_Find_Rel as Pos_Find_Rel_List

    UNWIND
      CASE
        WHEN Neg_Find_Rel = [] THEN [null]
        ELSE Neg_Find_Rel
      END AS Neg_Find_Rel_List

    // Calculate the sum of approximate frequency values for all negative findings for each disease, changing null to 0 when no relationships to negative findings exist
    WITH Disease, Disease_CUI, Positive_Findings, Pos_Find_CUIs, Negative_Findings, Neg_Find_CUIs, Pos_Find_Rel_List, sum(toFloat(COALESCE(Neg_Find_Rel_List.co_occurrence_probability, 0))) AS Sum_Neg_Find_Freq, collect(toFloat(Neg_Find_Rel_List.co_occurrence_probability)) AS Neg_Find_Freqs

    // Calculate the sum of approximate frequency values for all positive findings for each disease
    WITH Disease, Disease_CUI, Positive_Findings, Pos_Find_CUIs, Negative_Findings, Neg_Find_CUIs, sum(toFloat(Pos_Find_Rel_List.co_occurrence_probability)) AS Sum_Pos_Find_Freq, collect(toFloat(Pos_Find_Rel_List.co_occurrence_probability)) AS Pos_Find_Freqs, Sum_Neg_Find_Freq, Neg_Find_Freqs


    // Multiply the disease prevalence by the difference between the sum of frequencies of positive and negative findings
    RETURN Disease, Disease_CUI, Positive_Findings, Pos_Find_CUIs, Pos_Find_Freqs, Negative_Findings, Neg_Find_CUIs, Neg_Find_Freqs, (Sum_Pos_Find_Freq - Sum_Neg_Find_Freq) AS Probability

    ORDER BY Probability DESC
    LIMIT 10
    '''.format(cui_finding_list = cui_finding_list)
    data = session.run(query)
    mostLikelyRareDiseases = pd.DataFrame([dict(record) for record in data])
    
    return mostLikelyRareDiseases

In [16]:
# Test the rare disease search function
start_time = time.time()
test_result = DDx(NSCLC_entities)
print("Total runtime:", time.time() - start_time, "seconds")

test_result.head(5)

Total runtime: 2.1348352432250977 seconds


,Disease,Disease_CUI,Positive_Findings,Pos_Find_CUIs,Pos_Find_Freqs,Negative_Findings,Neg_Find_CUIs,Neg_Find_Freqs,Probability
0,Non-Small Cell Lung Carcinoma,C0007131,"[Carcinoma of lung, Malignant neoplasm of lung...","[C0684249, C0242379, C1839839]","[0.234027073390936, 0.424419323444471, 0.20367...",[],[],[0.334562653067215],0.527557
1,Malignant neoplasm of lung,C0242379,[Carcinoma of lung],[C0684249],[0.312874605729501],"[Non-Small Cell Lung Carcinoma, Sundowning]","[C0007131, C1142436]","[0.424419323444471, 0.212107155426606]",-0.323652
2,Neoplasm Metastasis,C0027627,[Carcinoma of lung],[C0684249],[0.394674065387146],[Neoplasm of uncertain or unknown behavior of ...,"[C0496956, C0006142, C0231218]","[0.2744218682836, 0.256387959133101, 0.2347106...",-0.370846
3,Carcinoma of lung,C0684249,[Malignant neoplasm of lung],[C0242379],[0.312874605729501],"[Neoplasm Metastasis, Chronic multifocal osteo...","[C0027627, C0410422, C0007131]","[0.394674065387146, 0.259410906606858, 0.23402...",-0.575237
4,Chronic multifocal osteomyelitis,C0410422,[Carcinoma of lung],[C0684249],[0.259410906606858],"[Shock, Cardiogenic, Stupor, Acute myocardial ...","[C0036980, C0085628, C0155626, C0521158]","[0.231270604756821, 0.228630197959932, 0.21346...",-0.614388


In [17]:
# Test the rare disease search function
start_time = time.time()
test_result = DDx(urosepsis_entities)
print("Total runtime:", time.time() - start_time, "seconds")

test_result.head(5)

Total runtime: 2.8185534477233887 seconds


,Disease,Disease_CUI,Positive_Findings,Pos_Find_CUIs,Pos_Find_Freqs,Negative_Findings,Neg_Find_CUIs,Neg_Find_Freqs,Probability
0,Sepsis due to urinary tract infection,C0149801,"[Actinic cheilitis, Flash pulmonary oedema, In...","[C0267026, C1168329, C3714514, C0264490, C0268...","[0.290588299369199, 0.222138304325536, 0.28636...",[],[],[0.368480229331998],2.408836
1,Embolism,C0013922,"[Acute respiratory failure, Flash pulmonary oe...","[C0264490, C1168329]","[0.237134365305756, 0.210483857472007]","[Pulmonary Embolism, gastrointestinal gas, Bro...","[C0034065, C0596601, C0006277, C0004626, C0748...","[0.322957956886218, 0.258178360249839, 0.22422...",-0.776008
2,Atrial Flutter,C0004239,[Oral candidiasis],[C0006849],[0.210285130377099],"[Cardiac Flutter, Edema of lower extremity, Pr...","[C0016385, C0239340, C0554309, C0876973]","[0.370518520253345, 0.265380349704386, 0.22006...",-0.848190
3,gastrointestinal gas,C0596601,[Actinic cheilitis],[C0267026],[0.303300320144778],"[Embolism, Hemolysis (disorder), Peripheral ed...","[C0013922, C0019054, C0085649, C0751781, C0003...","[0.258178360249839, 0.254293216233613, 0.24162...",-0.891586
4,Streptococcal Infections,C0038395,[Anxiety],[C0003467],[0.287381549196631],"[Supraventricular tachycardia, MAJOR AFFECTIVE...","[C0039240, C1839839, C0006277, C0687133, C1140...","[0.336150874737774, 0.289176739093851, 0.21920...",-0.968116


In [18]:
# Test the rare disease search function
start_time = time.time()
test_result = DDx(dCHF_entities)
print("Total runtime:", time.time() - start_time, "seconds")

test_result.head(5)

Total runtime: 3.923231363296509 seconds


,Disease,Disease_CUI,Positive_Findings,Pos_Find_CUIs,Pos_Find_Freqs,Negative_Findings,Neg_Find_CUIs,Neg_Find_Freqs,Probability
0,diastolic congestive heart failure,C2183328,[Natriuretic peptide.B prohormone N-Terminal:M...,"[C1316226, C1168329, C0235480, C0595862, C0546...","[0.251020279570385, 0.267430461475254, 0.31299...",[],[],[],4.996519
1,Mental Depression,C0011570,"[Signs and Symptoms, Respiratory]",[C0037090],[0.291840458960393],[Oozing (Hemorrhage)],[C1446325],[0.273771504859568],0.018069
2,Signs and Symptoms,C0037088,"[Flash pulmonary oedema, Vasodilation disorder...","[C1168329, C0595862, C0028754, C0018802]","[0.216936967859424, 0.225079374159778, 0.24938...","[Virus Diseases, Impaired health, Critical Ill...","[C0042769, C0684336, C0010340, C0085605]","[0.291727330680495, 0.275403118361964, 0.23818...",-0.137091
3,asthma with copd,C0865800,[Obesity],[C0028754],[0.315446915781628],"[Anemia due to blood loss, Chest Pain, Sleep A...","[C0948824, C0008031, C0520679]","[0.22269270288299, 0.214646541344892, 0.210932...",-0.332825
4,Myocardial Ischemia,C0151744,[Chronic back pain],[C0740418],[0.234950862879126],"[Acute Coronary Syndrome, Anemia, Macrocytic, ...","[C0948089, C0002886, C0019080, C4761312]","[0.241086030023517, 0.224928743827397, 0.21428...",-0.649413


In [23]:
# Test the rare disease search function
start_time = time.time()
test_result = DDx(bleed_out_entities)
print("Total runtime:", time.time() - start_time, "seconds")

test_result.head(5)

Total runtime: 2.2770369052886963 seconds


,Disease,Disease_CUI,Positive_Findings,Pos_Find_CUIs,Pos_Find_Freqs,Negative_Findings,Neg_Find_CUIs,Neg_Find_Freqs,Probability
0,Acute posthaemorrhagic anaemia,C0154298,"[bleeding/hemorrhage nos, Transient Ischemic A...","[C0497131, C0007787, C0014591, C0019080, C1135...","[0.237134365305756, 0.204020027632119, 0.22685...",[],[],[],1.129865
1,Transient Ischemic Attack,C0007787,"[Epistaxis, Heart Failure, Systolic]","[C0014591, C1135191]","[0.244708389348131, 0.262646867343228]","[Acute myocardial infarction, Dizziness, Acute...","[C0155626, C0012833, C0154298]","[0.221986317221007, 0.205243737758916, 0.20402...",-0.123895
2,Angina Pectoris,C0002962,[bleeding/hemorrhage nos],[C0497131],[0.200903982257697],"[Acute Coronary Syndrome, Hydrocephalus]","[C0948089, C0020255]","[0.263360836616665, 0.223284721342557]",-0.285742
3,Iron deficiency anemia secondary to chronic bl...,C0154286,[Epistaxis],[C0014591],[0.214495524212065],"[Malignant Neoplasms, Hemoptysis, Little's Dis...","[C0006826, C0019079, C0023882]","[0.214646541344892, 0.209002276005508, 0.20324...",-0.412402
4,Acute myocardial infarction,C0155626,[Transient Ischemic Attack],[C0007787],[0.221986317221007],[THYROID HORMONE PLASMA MEMBRANE TRANSPORT DEF...,"[C1861101, C0410422, C1761609]","[0.241622675849386, 0.213465540173336, 0.20090...",-0.434006


In [15]:
    cui_prob_list = ['C1405315']
    # Get NLP-derived problem matches
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"]
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r, c
    RETURN c.cui AS `Known_CUI`, c.term AS `Known_Problem`, ord.cui AS `CUI`, ord.term AS `Problem`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    likely_comorbidities_NLP = pd.DataFrame([dict(record) for record in data])
    
    # Get ICD-coded problem matches
    query = '''
    MATCH p=(ord:D_Icd_Diagnoses)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} 
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r, c
    RETURN c.cui AS `Known_CUI`, c.term AS `Known_Problem`, ord.icd9_code AS `CUI`, ord.long_title AS `Problem`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    likely_comorbidities_ICD = pd.DataFrame([dict(record) for record in data])
    
    # Combine the NLP-derived and ICD-coded dataframes for output
    likely_comorbidities = likely_comorbidities_NLP.append(likely_comorbidities_ICD)
    try:
        likely_comorbidities.sort_values(by='Score', ascending=False, inplace=True)
        likely_comorbidities.head(10)
    except:
        likely_comorbidities = pd.DataFrame(columns = ['Known_CUI', 'Known_Problem', 'CUI', 'Problem', 'Score'])
        likely_comorbidities.iloc[0:0] = 'No results'
        print(likely_comorbidities.head(10))

Empty DataFrame
Columns: [Known_CUI, Known_Problem, CUI, Problem, Score]
Index: []


In [4]:
# Get a list of autocomplete terms
startingtext = 'Res'
query = '''
MATCH (c1:Concept)
WHERE c1.term STARTS WITH '{startingtext}' AND c1.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"] 
RETURN DISTINCT(c1.cui) AS `Known_CUI`, c1.term AS `Known_Problem`
'''.format(startingtext=startingtext)
data = session.run(query)
autocomplete_problems_prompts = pd.DataFrame([dict(record) for record in data])
autocomplete_problems_prompts

,Known_CUI,Known_Problem
0,C4046002,Research Related Injuries
1,C4046002,Research-Related Injuries
2,C4046002,Research-Related Injury
3,C0862970,Resectable Bile Duct Adenocarcinoma
4,C0854792,Resectable Bile Duct Cancer
...,...,...
985,C3281186,Restrictive ventilation disorder due to severe...
986,C0036351,Restzustand
987,C0036351,Restzustand (schizophrenic)
988,C0036351,"Restzustand, schizophrenic"


In [94]:
    cui_prob_list = ['C1565489', 'C0085762']
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"]
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r, c
    RETURN c.cui AS `Known_CUI`, c.term AS `Known_Problem`, ord.cui AS `CUI`, ord.term AS `Problem`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    likely_comorbidities_NLP = pd.DataFrame([dict(record) for record in data])
    likely_comorbidities_NLP

,Known_CUI,Known_Problem,CUI,Problem,Score
0,C0085762,Alcohol abuse,C0220982,Ketoacidosis,0.61598
1,C0085762,Alcohol abuse,C0149521,"Pancreatitis, Chronic",0.56788
2,C0085762,Alcohol abuse,C0151723,Hypomagnesemia,0.51528
3,C0085762,Alcohol abuse,C0236663,Alcohol withdrawal syndrome,0.51037
4,C0085762,Alcohol abuse,C1963823,Hyperlipasaemia,0.46902
5,C0085762,Alcohol abuse,C0438696,Suicidal,0.46201
6,C0085762,Alcohol abuse,C0747752,Polysubstance abuse,0.45843
7,C0085762,Alcohol abuse,C0162316,Iron deficiency anemia,0.44179
8,C0085762,Alcohol abuse,C3159311,BORNHOLM EYE DISEASE,0.42945
9,C0085762,Alcohol abuse,C0232498,Abdominal tenderness,0.42369


In [18]:
# get a CSV of all problems in the database that have enough data to potentially return robust results
query = '''
MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
WITH round(r.co_occurrance_probability, 5)*1000 AS Score, ord, r, c
WHERE Score > 20
RETURN distinct(c.term) AS Problem, c.cui'''
data = session.run(query)

# Convert the neo4j object into a dataframe
common_problems_df = pd.DataFrame([dict(record) for record in data])

In [21]:
common_problems_df.sort_values(by='Problem', inplace=True)
common_problems_df.Problem.to_list()

['Abnormally opaque structure (morphologic abnormality)',
 'Acidosis',
 'Acidosis, Lactic',
 'Acidosis, Respiratory',
 'Actinic cheilitis',
 'Acute respiratory failure',
 'Aggressive Periodontitis',
 'Agitation',
 'Alcohol Withdrawal Delirium',
 'Alcohol abuse',
 'Alcohol withdrawal syndrome',
 'Alkalosis, Respiratory',
 'Aneurysm',
 'Apnea',
 'Ascites',
 'Back Pain',
 'Blood Clot',
 'Bronchospasm',
 'Cardiac Arrhythmia',
 'Cerebrovascular accident',
 'Chest Pain',
 'Cholangiocarcinoma',
 'Chronic Kidney Diseases',
 'Chronic back pain',
 'Confusion',
 'Contraction (finding)',
 'Coughing',
 'Dentatorubral-Pallidoluysian Atrophy',
 'Desmoplastic infantile ganglioglioma',
 'Diabetes Mellitus, Non-Insulin-Dependent',
 'Diabetic Ketoacidosis',
 'Diastolic dysfunction',
 'Disease',
 'Disease Progression',
 'Dry skin',
 'Dyspnea',
 'Encephalitis',
 'Epilepsy',
 'Esophageal Varices',
 'Fatigue',
 'Febrile Neutropenia',
 'Flare',
 'Fluid overload',
 'Gastrointestinal Hemorrhage',
 'Gastroparesi

In [26]:
# Trim out problems that seem to be spurious
spurious = ['Abnormally opaque structure (morphologic abnormality)', 'Actinic cheilitis', 'Aggressive Periodontitis', 'Dentatorubral-Pallidoluysian Atrophy', 'Desmoplastic infantile ganglioglioma',  'Disease', 'Disease Progression', 'Flare', 'Obstruction', 'POPLITEAL PTERYGIUM SYNDROME, LETHAL TYPE',  'Pathological Dilatation', 'Pathology processes', 'Sore to touch', 'Xanthomatosis, Cerebrotendinous',  'diastolic congestive heart failure', 'gastrointestinal gas', 'physiologic resolution']
common_problems_df = common_problems_df[~common_problems_df.Problem.isin(spurious)]

In [27]:
# Write out to CSV
common_problems_df.to_csv('common_problems.csv', index=False, encoding='utf-8')

## Modify existing functions for the new data model
### Input a list of CUIs for known problems, output possible additional problems with their CUIs and odds ratios in a dataframe
Note: This query relies on the OCCURS_WITH relationship, which was created in the [Virtual_Population](Documents/P2_Prototype/Virtual_Population.ipynb) notebook under the section "Using Conditional Probability instead of Odds Ratio."

In [12]:
def PotentialComorbidities(cui_prob_list):
    
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Pathologic Function']", "['Disease or Syndrome']", "['Mental or Behavioral Dysfunction']", "['Sign or Symptom']", "['Injury or Poisoning']", "['Neoplastic Process']"]
    WITH round(r.co_occurrance_probability, 5) AS Score, ord, r, c
    RETURN c.cui AS `Known_CUI`, c.term AS `Known_Problem`, ord.cui AS `Likely_CUI`, ord.term AS `Likely_Problem`, Score
    ORDER BY r.co_occurrance_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    likely_comorbidities = pd.DataFrame([dict(record) for record in data])
    
    return likely_comorbidities

In [13]:
# Test the function
start_time = time.time()

cui_prob_list = ['C1565489', 'C0085762']
PotentialComorbidities = PotentialComorbidities(cui_prob_list)

print("Total runtime:", time.time() - start_time, "seconds")
PotentialComorbidities

Total runtime: 0.0643320083618164 seconds


,Known_CUI,Known_Problem,Likely_CUI,Likely_Problem,Score
0,C0085762,Alcohol abuse,C0162316,Iron deficiency anemia,None
1,C0085762,Alcohol abuse,C0948268,Hemodynamic instability,None
2,C0085762,Alcohol abuse,C0394996,Acute alcoholic intoxication,None
3,C0085762,Alcohol abuse,C4761312,Nutritional Deficiency,None
4,C0085762,Alcohol abuse,C0220982,Ketoacidosis,None
5,C0085762,Alcohol abuse,C0677042,Pathology processes,None
6,C0085762,Alcohol abuse,C0149521,"Pancreatitis, Chronic",None
7,C0085762,Alcohol abuse,C0151723,Hypomagnesemia,None
8,C0085762,Alcohol abuse,C0338908,Mixed anxiety and depressive disorder,None
9,C1565489,Renal Insufficiency,C2826321,Refractory Thrombocytopenia,None


### Input 1 problem, output the most likely orders
Note: This query relies on the OCCURS_WITH relationship, which was created in the [Virtual_Population](Documents/P2_Prototype/Virtual_Population.ipynb) notebook under the section "Using Conditional Probability instead of Odds Ratio."

In [9]:
def LikelyOrders(cui_prob_list):
    
    # Find prescriptions associated with the input problem    
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Clinical Drug']"]
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r
    RETURN ord.cui AS `Code`, ord.term AS `Order`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_rx = pd.DataFrame([dict(record) for record in data]).head(10)
    
    # Find abnormal labs associated with the input problem
    query = '''
    MATCH p=(ord:Concept)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list} AND ord.semantic_type IN ["['Clinical Attribute']"]
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r
    RETURN ord.cui AS `Code`, ord.description AS `Order`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_lab = pd.DataFrame([dict(record) for record in data]).head(10)
    
    # Find procedures associated with the input problem
    query = '''
    MATCH p=(ord:D_Icd_Procedures)-[r:OCCURS_WITH]->(c:Concept) 
    WHERE c.cui IN {cui_prob_list}
    WITH round(r.co_occurrence_probability, 5) AS Score, ord, r
    RETURN ord.icd9_code AS `Code`, ord.long_title AS `Order`, Score
    ORDER BY r.co_occurrence_probability DESC
    LIMIT 10
    '''.format(cui_prob_list=cui_prob_list)
    data = session.run(query)
    orders_likely_procedure = pd.DataFrame([dict(record) for record in data]).head(10)
    
    return orders_likely_rx, orders_likely_lab, orders_likely_procedure

In [ ]:
MATCH (source:Concept {cui:'C0022661'})
MATCH (destination:Concept {cui:'C0975120'})
CALL gds.alpha.shortestPath.stream({
    startNode: source,
    endNode: destination,
    nodeProjection: 
})

In [26]:
start_time = time.time()

cui_prob_list = ['C0022661']
orders_likely_rx, orders_likely_lab, orders_likely_procedure = LikelyOrders(cui_prob_list)

print("Total runtime:", time.time() - start_time, "seconds")

Total runtime: 0.023564577102661133 seconds


In [27]:
orders_likely_rx

,Code,Order,Score
0,C0975120,calcitriol 0.00025 MG Oral Capsule,0.62000
1,C1967412,sevelamer carbonate 800 MG Oral Tablet [Renvela],0.60450
2,C1951501,sodium polystyrene sulfonate 250 MG/ML Oral Su...,0.58420
3,C0706911,1 ML epoetin alfa 4000 UNT/ML Injection [Procrit],0.44712
4,C4082775,150 ML Glucose 50 MG/ML Injection,0.42971
5,C0710995,purified protein derivative of tuberculin 50 U...,0.39966
6,C0691839,sodium bicarbonate 650 MG Oral Tablet,0.37536
7,C0978650,midodrine hydrochloride 5 MG Oral Tablet,0.35985
8,C1169852,piperacillin 2000 MG / tazobactam 250 MG Injec...,0.35321
9,C0975350,cefepime 1000 MG Injection,0.33979


In [28]:
orders_likely_lab

,Code,Order,Score
0,C1544491,Macrophage in Ascites,0.77165
1,C0549816,Lymphocytes in Other Body Fluid,0.76913
2,C0942476,Polys in Other Body Fluid,0.71643
3,C0942444,Lymphocytes in Ascites,0.70352
4,C0942425,"RBC, Ascites in Ascites",0.68548
5,C0945358,"WBC, Ascites in Ascites",0.67514
6,C0942450,Monocytes in Ascites,0.66127
7,C0942478,Polys in Ascites,0.64904
8,C0365034,Protein/Creatinine Ratio in Urine,0.63874
9,C0368042,pH in Urine,0.59096


In [29]:
orders_likely_procedure

,Code,Order,Score
0,3995,Hemodialysis,0.75877
1,3895,Venous catheterization for renal dialysis,0.70488
2,4513,Other endoscopy of small intestine,0.26072
3,3324,Closed [endoscopic] biopsy of bronchus,0.24204
4,3893,"Venous catheterization, not elsewhere classified",0.22290
5,9904,Transfusion of packed cells,0.22250
6,9604,Insertion of endotracheal tube,0.20985


In [25]:
orders_likely_rx['Type'] = 'Prescription'
orders_likely_lab['Type'] = 'Lab'
orders_likely_procedure['Type'] = 'Procedure'
AssocOrders = pd.concat([orders_likely_rx, orders_likely_lab, orders_likely_procedure])
AssocOrders.sort_values('Score', ascending = False, inplace = True)
AssocOrders = AssocOrders.head(10)
AssocOrders

,Code,Order,Score,Type
0,C1544491,Macrophage in Ascites,0.77165,Lab
1,C0549816,Lymphocytes in Other Body Fluid,0.76913,Lab
0,3995,Hemodialysis,0.75877,Procedure
2,C0942476,Polys in Other Body Fluid,0.71643,Lab
1,3895,Venous catheterization for renal dialysis,0.70488,Procedure
3,C0942444,Lymphocytes in Ascites,0.70352,Lab
4,C0942425,"RBC, Ascites in Ascites",0.68548,Lab
5,C0945358,"WBC, Ascites in Ascites",0.67514,Lab
6,C0942450,Monocytes in Ascites,0.66127,Lab
7,C0942478,Polys in Ascites,0.64904,Lab
